In [1]:
from signals_lab.combined_processor import CombinedProcessor
from signals_lab.eeg_config import EEGConfig
from torch.utils.data import DataLoader, random_split
from moabb.datasets import Lee2019_SSVEP
import pytorch_lightning as pl
import pandas as pd

matplotlib - matplotlib data path: /mnt/c/Users/erict/Development/git/Torchsig-VQ-Transformer/.venv/lib/python3.12/site-packages/matplotlib/mpl-data
matplotlib - CONFIGDIR=/home/eric/.config/matplotlib
matplotlib - interactive is False
matplotlib - platform is linux
matplotlib - CACHEDIR=/home/eric/.cache/matplotlib
matplotlib.font_manager - Using fontManager instance from /home/eric/.cache/matplotlib/fontlist-v390.json
h5py._conv - Creating converter from 7 to 5
h5py._conv - Creating converter from 5 to 7
h5py._conv - Creating converter from 7 to 5
h5py._conv - Creating converter from 5 to 7


In [2]:
lee_config = EEGConfig(
    dataset=Lee2019_SSVEP(),
    num_channels=62,
    sample_rate=1000,
    subject_range=(1, 2),
    filter_range=(5, 95),
    channel_range=("Fp1", "PO4"),
)

combined_processor = CombinedProcessor(configs=[lee_config])

signals_lab.combined_processor - Creating CombinedProcessor and populating fields.
signals_lab.eeg_processor - Calculating generator length.
signals_lab.eeg_processor - Processing subject 1.
signals_lab.eeg_processor - Processing session 0.
signals_lab.eeg_processor - Processing run 1train.
signals_lab.eeg_processor - Filtering data from 5 to 95 Hz


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 5 - 95 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 95.00 Hz
- Upper transition bandwidth: 23.75 Hz (-6 dB cutoff frequency: 106.88 Hz)
- Filter length: 1651 samples (1.651 s)

Using filter length: 1651
Smart-padding with:  1650 samples on each edge
FFT block length:   16384
Got 1 parallel jobs after requesting None


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.5s
signals_lab.eeg_processor - Extracting channels Fp1 to PO4.
signals_lab.eeg_processor - Reshaping data into segments of 512 samples per channel.
signals_lab.eeg_processor - Extracting 3048 segments with zero padding for the last one if needed.
signals_lab.eeg_processor - Zero padding segment 3048: original size (16, 62).
signals_lab.eeg_processor - Processing session 1.
signals_lab.eeg_processor - Processing run 1train.
signals_lab.eeg_processor - Filtering data from 5 to 95 Hz


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 5 - 95 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 95.00 Hz
- Upper transition bandwidth: 23.75 Hz (-6 dB cutoff frequency: 106.88 Hz)
- Filter length: 1651 samples (1.651 s)

Using filter length: 1651
Smart-padding with:  1650 samples on each edge
FFT block length:   16384
Got 1 parallel jobs after requesting None


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s
signals_lab.eeg_processor - Extracting channels Fp1 to PO4.
signals_lab.eeg_processor - Reshaping data into segments of 512 samples per channel.
signals_lab.eeg_processor - Extracting 2536 segments with zero padding for the last one if needed.
signals_lab.eeg_processor - Zero padding segment 2536: original size (80, 62).
signals_lab.combined_processor - Creating segment iterator.


In [3]:
train_set, val_set = random_split(combined_processor, [.8, .2])
train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
val_loader = DataLoader(val_set, batch_size=8, shuffle=False)

In [2]:
test_set = Lee2019_SSVEP()
subjects_data = test_set.get_data(subjects=[1])
for subject, data in subjects_data.items():
    for run in data.items():
        for trial in run[1].items():
            hello = pd.DataFrame(run[1][trial[0]].to_data_frame())
            print(hello)

            print(run[1][trial[0]].info['ch_names'])  # List of channel names
            print(run[1][trial[0]].info['sfreq'])  # Sampling frequency
            print(run[1][trial[0]].info['nchan'])  # Number of channels

            # View all metadata
            print(run[1][trial[0]].info)


             time         Fp1         Fp2          F7          F3          Fz  \
0           0.000   42.400002   28.700001    2.900000   35.000000  156.100006   
1           0.001   46.099998   40.099998    9.300000   39.799999  157.500000   
2           0.002   43.000000   40.000000    9.500000   36.299999  153.899994   
3           0.003   40.900002   35.000000    8.100000   35.599998  152.600006   
4           0.004   41.700001   39.500000   10.600000   40.700001  153.899994   
...           ...         ...         ...         ...         ...         ...   
1560075  1560.075  387.700012  439.100006  235.500000  254.100006  252.600006   
1560076  1560.076  382.899994  436.700012  228.199997  249.800003  250.300003   
1560077  1560.077  380.100006  430.399994  218.800003  243.600006  245.000000   
1560078  1560.078  394.399994  437.100006  234.399994  255.100006  257.299988   
1560079  1560.079  405.200012  452.299988  241.000000  263.000000  268.200012   

                 F4        